# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Mar 2 2023 10:54AM,258534,19,60021820,"GCH Granules USA, Inc.",Released
1,Mar 2 2023 10:54AM,258534,19,60021826,"GCH Granules USA, Inc.",Released
2,Mar 2 2023 10:54AM,258534,19,60021827,"GCH Granules USA, Inc.",Released
3,Mar 2 2023 10:50AM,258532,10,PRF-54509,Bio-PRF,Released
4,Mar 2 2023 10:50AM,258532,10,PRF-54514,Bio-PRF,Released
5,Mar 2 2023 10:50AM,258532,10,PRF-54518,Bio-PRF,Released
6,Mar 2 2023 10:50AM,258532,10,PRF-54578,Bio-PRF,Released
7,Mar 2 2023 10:50AM,258532,10,PRF-54583,Bio-PRF,Released
8,Mar 2 2023 10:49AM,258531,10,9220929,Eywa Pharma Inc.,Released
9,Mar 2 2023 10:49AM,258531,10,9220930,Eywa Pharma Inc.,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
25,258530,Released,5
26,258531,Released,10
27,258532,Released,5
28,258533,Released,4
29,258534,Released,3


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
258530,NaN,NaN,5.0
258531,NaN,NaN,10.0
258532,NaN,NaN,5.0
258533,NaN,NaN,4.0
258534,NaN,NaN,3.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
258460,0.0,15.0,10.0
258461,0.0,15.0,18.0
258478,0.0,1.0,0.0
258483,0.0,1.0,0.0
258487,0.0,1.0,0.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
258460,0,15,10
258461,0,15,18
258478,0,1,0
258483,0,1,0
258487,0,1,0


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,258460,0,15,10
1,258461,0,15,18
2,258478,0,1,0
3,258483,0,1,0
4,258487,0,1,0


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,258460,,15,10
1,258461,,15,18
2,258478,,1,
3,258483,,1,
4,258487,,1,


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Mar 2 2023 10:54AM,258534,19,"GCH Granules USA, Inc."
3,Mar 2 2023 10:50AM,258532,10,Bio-PRF
8,Mar 2 2023 10:49AM,258531,10,Eywa Pharma Inc.
18,Mar 2 2023 10:48AM,258533,19,"GCH Granules USA, Inc."
22,Mar 2 2023 10:40AM,258530,19,"GCH Granules USA, Inc."
27,Mar 2 2023 10:40AM,258529,15,"Alliance Pharma, Inc."
32,Mar 2 2023 10:30AM,258528,19,"GCH Granules USA, Inc."
37,Mar 2 2023 10:23AM,258527,10,"ClearSpec, LLC"
38,Mar 2 2023 10:21AM,258526,19,"GCH Granules USA, Inc."
43,Mar 2 2023 10:11AM,258523,19,"GCH Granules USA, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Mar 2 2023 10:54AM,258534,19,"GCH Granules USA, Inc.",,,3
1,Mar 2 2023 10:50AM,258532,10,Bio-PRF,,,5
2,Mar 2 2023 10:49AM,258531,10,Eywa Pharma Inc.,,,10
3,Mar 2 2023 10:48AM,258533,19,"GCH Granules USA, Inc.",,,4
4,Mar 2 2023 10:40AM,258530,19,"GCH Granules USA, Inc.",,,5
5,Mar 2 2023 10:40AM,258529,15,"Alliance Pharma, Inc.",,,5
6,Mar 2 2023 10:30AM,258528,19,"GCH Granules USA, Inc.",,,5
7,Mar 2 2023 10:23AM,258527,10,"ClearSpec, LLC",,,1
8,Mar 2 2023 10:21AM,258526,19,"GCH Granules USA, Inc.",,,5
9,Mar 2 2023 10:11AM,258523,19,"GCH Granules USA, Inc.",,,5


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Mar 2 2023 10:54AM,258534,19,"GCH Granules USA, Inc.",3,,
1,Mar 2 2023 10:50AM,258532,10,Bio-PRF,5,,
2,Mar 2 2023 10:49AM,258531,10,Eywa Pharma Inc.,10,,
3,Mar 2 2023 10:48AM,258533,19,"GCH Granules USA, Inc.",4,,
4,Mar 2 2023 10:40AM,258530,19,"GCH Granules USA, Inc.",5,,
5,Mar 2 2023 10:40AM,258529,15,"Alliance Pharma, Inc.",5,,
6,Mar 2 2023 10:30AM,258528,19,"GCH Granules USA, Inc.",5,,
7,Mar 2 2023 10:23AM,258527,10,"ClearSpec, LLC",1,,
8,Mar 2 2023 10:21AM,258526,19,"GCH Granules USA, Inc.",5,,
9,Mar 2 2023 10:11AM,258523,19,"GCH Granules USA, Inc.",5,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Mar 2 2023 10:54AM,258534,19,"GCH Granules USA, Inc.",3,,
1,Mar 2 2023 10:50AM,258532,10,Bio-PRF,5,,
2,Mar 2 2023 10:49AM,258531,10,Eywa Pharma Inc.,10,,
3,Mar 2 2023 10:48AM,258533,19,"GCH Granules USA, Inc.",4,,
4,Mar 2 2023 10:40AM,258530,19,"GCH Granules USA, Inc.",5,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Mar 2 2023 10:54AM,258534,19,"GCH Granules USA, Inc.",3.0,NaN,NaN
1,Mar 2 2023 10:50AM,258532,10,Bio-PRF,5.0,NaN,NaN
2,Mar 2 2023 10:49AM,258531,10,Eywa Pharma Inc.,10.0,NaN,NaN
3,Mar 2 2023 10:48AM,258533,19,"GCH Granules USA, Inc.",4.0,NaN,NaN
4,Mar 2 2023 10:40AM,258530,19,"GCH Granules USA, Inc.",5.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Mar 2 2023 10:54AM,258534,19,"GCH Granules USA, Inc.",3.0,0.0,0.0
1,Mar 2 2023 10:50AM,258532,10,Bio-PRF,5.0,0.0,0.0
2,Mar 2 2023 10:49AM,258531,10,Eywa Pharma Inc.,10.0,0.0,0.0
3,Mar 2 2023 10:48AM,258533,19,"GCH Granules USA, Inc.",4.0,0.0,0.0
4,Mar 2 2023 10:40AM,258530,19,"GCH Granules USA, Inc.",5.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2326571,35.0,4.0,0.0
12,258487,0.0,1.0,0.0
15,1033945,36.0,30.0,0.0
19,2068188,37.0,5.0,4.0
20,517014,2.0,0.0,0.0
22,258498,1.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2326571,35.0,4.0,0.0
1,12,258487,0.0,1.0,0.0
2,15,1033945,36.0,30.0,0.0
3,19,2068188,37.0,5.0,4.0
4,20,517014,2.0,0.0,0.0
5,22,258498,1.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,35.0,4.0,0.0
1,12,0.0,1.0,0.0
2,15,36.0,30.0,0.0
3,19,37.0,5.0,4.0
4,20,2.0,0.0,0.0
5,22,1.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,35.0
1,12,Released,0.0
2,15,Released,36.0
3,19,Released,37.0
4,20,Released,2.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,19,20,22
Status,,,,,,
Completed,0.0,0.0,0.0,4.0,0.0,0.0
Executing,4.0,1.0,30.0,5.0,0.0,0.0
Released,35.0,0.0,36.0,37.0,2.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,19,20,22
0,Completed,0.0,0.0,0.0,4.0,0.0,0.0
1,Executing,4.0,1.0,30.0,5.0,0.0,0.0
2,Released,35.0,0.0,36.0,37.0,2.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,19,20,22
0,Completed,0.0,0.0,0.0,4.0,0.0,0.0
1,Executing,4.0,1.0,30.0,5.0,0.0,0.0
2,Released,35.0,0.0,36.0,37.0,2.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()